# 라이브러리 불러오기

In [15]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

# Text Spliter
- Text Spliter는 특정 기준을 통해 텍스트를 청크로 나누는 모듈
- CharacterTextSpliter는 특정 구분자를 기준으로 분할
- 토큰 제한 이슈를 우회하여 context 학습 가능

# CharacterTextSpliter
- 가장 간당한 텍스트 분할기
- 특정 구분자를 기준으로 텍스트를 여러개로 분할

In [12]:
!ls data

'[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf'
'최종 필터링 데이터 (호텔, 지형) - 시트3 (1).csv'
 BOK이슈노트제2023-26호.pdf
 해커톤.docx
'[복지이슈 FOCUS 15ȣ] 경기도 극저신용대출심사모형 개발을 위한 국내 신용정보 활용가능성 탐색.pdf'
 국문_MS_Word_파일_DOC_202402.docx
 state_of_the_union.txt
 titanic.csv
 Transformer_paper.pdf


In [14]:
with open('./data/state_of_the_union.txt') as f:
    state_of_the_union = f.read()

In [20]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len,
)

In [21]:
texts = text_splitter.split_text(state_of_the_union)

Created a chunk of size 1163, which is longer than the specified 1000
Created a chunk of size 1015, which is longer than the specified 1000


In [23]:
print(texts[0])
print('-'*100)
print(texts[1])
print('-'*100)
print(texts[2])
print('-'*100)

Madame Speaker, Vice President Biden, members of Congress, distinguished guests, and fellow Americans:

Our Constitution declares that from time to time, the president shall give to Congress information about the state of our union. For 220 years, our leaders have fulfilled this duty. They have done so during periods of prosperity and tranquility. And they have done so in the midst of war and depression; at moments of great strife and great struggle.
----------------------------------------------------------------------------------------------------
It's tempting to look back on these moments and assume that our progress was inevitable, that America was always destined to succeed. But when the Union was turned back at Bull Run and the Allies first landed at Omaha Beach, victory was very much in doubt. When the market crashed on Black Tuesday and civil rights marchers were beaten on Bloody Sunday, the future was anything but certain. These were times that tested the courage of our convi

- chunk size 확인

In [24]:
char_list = []
for i in range(len(texts)):
    char_list.append(len(texts[i]))
print(char_list)

[454, 707, 930, 848, 896, 818, 846, 892, 813, 721, 730, 919, 996, 926, 839, 900, 850, 529, 614, 493, 582, 746, 1163, 878, 483, 607, 740, 419, 902, 794, 714, 416, 894, 917, 934, 711, 548, 506, 784, 948, 619, 669, 510, 1015, 714, 527, 701, 597, 436, 750, 558, 992, 920, 786, 790]


In [25]:
text_splitter.create_documents([state_of_the_union])

Created a chunk of size 1163, which is longer than the specified 1000
Created a chunk of size 1015, which is longer than the specified 1000


[Document(page_content='Madame Speaker, Vice President Biden, members of Congress, distinguished guests, and fellow Americans:\n\nOur Constitution declares that from time to time, the president shall give to Congress information about the state of our union. For 220 years, our leaders have fulfilled this duty. They have done so during periods of prosperity and tranquility. And they have done so in the midst of war and depression; at moments of great strife and great struggle.'),
 Document(page_content="It's tempting to look back on these moments and assume that our progress was inevitable, that America was always destined to succeed. But when the Union was turned back at Bull Run and the Allies first landed at Omaha Beach, victory was very much in doubt. When the market crashed on Black Tuesday and civil rights marchers were beaten on Bloody Sunday, the future was anything but certain. These were times that tested the courage of our convictions and the strength of our union. And despit

- 한글 문서로 테스트

In [27]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('./data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf')
pages = loader.load_and_split()

In [28]:
len(pages)

18

In [29]:
print(pages[1].page_content)

| 2 | CIS이슈리포트 2022-2 호 ▶혁신성장 ICT 산업의 정책금융 공급규모 및 공급속도를 종합적으로 분석한 결과, 차세대무선통신미디어 , 
능동형컴퓨팅 (이상 정보통신 테마), 차세대반도체 (전기전자 테마) 및 객체탐지 (센서측정 테마) 기술분야로 
혁신성장 정책금융이 집중되고 있음
[ICT 산업 내 주요 기술분야 혁신성장 정책금융 공급 현황]
                                                            (단위: 억 원, %)
테마
(대분류 )주요 기술분야
(중분류 )정책금융 공급규모 연평균 공급액 
증가율 (%)테마 내 공급 
점유율 (%) 2017 년 말 2021 년 말
정보통신차세대무선통신미디어 7,820 27,865 37.4 45.1
능동형컴퓨팅 352 16,032 159.8 10.1
전기전자 차세대반도체 12,019 53,779 45.4 58.5
센서측정 객체탐지 1,278 6,711 51.4 48.5
▶주요 기술분야별 세부 품목단위로는 5G 이동통신시스템 , 인공지능 (AI), 시스템반도체 및 스마트센서 에 
정책금융 공급량이 높은 것으로 확인됨
￮정부가 미래 먹거리산업으로 선정한 인공지능 (AI)의 미래성장율 (CAGR: 41.0%) 이 가장 높으며 , 시장규모는  
시스템반도체 (3,833.8 억 달러, 2025 년)가 가장 큰 것으로 분석됨
￮4대 품목은 공통적으로 수요기반이 크고, 각국 정부가 중점적으로 육성을 지원하고 있어 시장이 지속 
성장할 것으로 전망되나 , 원천기술 미확보 및 높은 해외 의존도가 약점으로 지적되어 국내 기업의 경쟁력  
강화가 시급한 것으로 평가됨
[혁신성장 ICT 주요 품목 시장전망 ]
                                                            (단위: 억 달러, %)
주요 기술분야
(중분류 )주요 품목
(소분류 )시장규모 전망시장 촉진·저해요인2020 년 2025년(E) CAGR(%)
차세대
무선통신미디어5G이

In [31]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 1000,
    chunk_overlap = 100,
    length_function = len,
)

In [32]:
texts = text_splitter.split_documents(pages)

- chunk size 확인

In [36]:
char_list = []
for i in range(len(texts)):
    char_list.append(len(texts[i].page_content))
print(char_list)

[820, 989, 576, 993, 254, 954, 371, 742, 669, 964, 739, 556, 963, 485, 971, 468, 986, 231, 544, 973, 332, 975, 205, 969, 460, 962, 92, 997, 82, 944, 197]


In [38]:
print(texts[1].page_content)

| 2 | CIS이슈리포트 2022-2 호 ▶혁신성장 ICT 산업의 정책금융 공급규모 및 공급속도를 종합적으로 분석한 결과, 차세대무선통신미디어 , 
능동형컴퓨팅 (이상 정보통신 테마), 차세대반도체 (전기전자 테마) 및 객체탐지 (센서측정 테마) 기술분야로 
혁신성장 정책금융이 집중되고 있음
[ICT 산업 내 주요 기술분야 혁신성장 정책금융 공급 현황]
                                                            (단위: 억 원, %)
테마
(대분류 )주요 기술분야
(중분류 )정책금융 공급규모 연평균 공급액 
증가율 (%)테마 내 공급 
점유율 (%) 2017 년 말 2021 년 말
정보통신차세대무선통신미디어 7,820 27,865 37.4 45.1
능동형컴퓨팅 352 16,032 159.8 10.1
전기전자 차세대반도체 12,019 53,779 45.4 58.5
센서측정 객체탐지 1,278 6,711 51.4 48.5
▶주요 기술분야별 세부 품목단위로는 5G 이동통신시스템 , 인공지능 (AI), 시스템반도체 및 스마트센서 에 
정책금융 공급량이 높은 것으로 확인됨
￮정부가 미래 먹거리산업으로 선정한 인공지능 (AI)의 미래성장율 (CAGR: 41.0%) 이 가장 높으며 , 시장규모는  
시스템반도체 (3,833.8 억 달러, 2025 년)가 가장 큰 것으로 분석됨
￮4대 품목은 공통적으로 수요기반이 크고, 각국 정부가 중점적으로 육성을 지원하고 있어 시장이 지속 
성장할 것으로 전망되나 , 원천기술 미확보 및 높은 해외 의존도가 약점으로 지적되어 국내 기업의 경쟁력  
강화가 시급한 것으로 평가됨
[혁신성장 ICT 주요 품목 시장전망 ]
                                                            (단위: 억 달러, %)
주요 기술분야
(중분류 )주요 품목
(소분류 )시장규모 전망시장 촉진·저해요인2020 년 2025년(E) CAGR(%)
차세대


# RecursiveCharacterTextSplit
- RecursiveCharacterTextSplit 으,ㄴ 재귀적으로 문서를 분할
- 먼저 "\n\n"(줄바꿈)을 기준으로 분할 하고
- 이렇게 나눈 청크가 여전히 너무 클 경우 "\n"(문장 단위)을 기준으로 분할
- 이렇게 해서도 청크가 충분히 작아지지 않는 다면, 단어 위주로 다르게 됨
- 최대한 문장의 의미를 보존하면서 분할 가능하게 해줌
- 따라서 따라서 많이 사용됨

In [46]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len,
)

In [47]:
texts = text_splitter.create_documents([state_of_the_union])

In [48]:
print(texts[0])
print('-'*100)
print(texts[1])
print('-'*100)
print(texts[2])
print('-'*100)

page_content='Madame Speaker, Vice President Biden, members of Congress, distinguished guests, and fellow Americans:\n\nOur Constitution declares that from time to time, the president shall give to Congress information about the state of our union. For 220 years, our leaders have fulfilled this duty. They have done so during periods of prosperity and tranquility. And they have done so in the midst of war and depression; at moments of great strife and great struggle.'
----------------------------------------------------------------------------------------------------
page_content="It's tempting to look back on these moments and assume that our progress was inevitable, that America was always destined to succeed. But when the Union was turned back at Bull Run and the Allies first landed at Omaha Beach, victory was very much in doubt. When the market crashed on Black Tuesday and civil rights marchers were beaten on Bloody Sunday, the future was anything but certain. These were times that 

- chunk size 확인 : 1000이 넘는 것이 하나도 없음

In [49]:
char_list = []
for i in range(len(texts)):
    char_list.append(len(texts[i].page_content))
print(char_list)

[454, 707, 930, 848, 896, 818, 846, 892, 813, 721, 730, 576, 841, 939, 980, 820, 936, 585, 529, 614, 493, 582, 746, 995, 365, 878, 483, 607, 740, 419, 902, 794, 714, 416, 894, 917, 934, 711, 548, 506, 784, 948, 619, 669, 510, 996, 216, 714, 527, 701, 597, 436, 750, 558, 992, 920, 786, 949]


- 한글 문서로 테스트

In [50]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('./data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf')
pages = loader.load_and_split()

In [56]:
print(pages[1].page_content)

| 2 | CIS이슈리포트 2022-2 호 ▶혁신성장 ICT 산업의 정책금융 공급규모 및 공급속도를 종합적으로 분석한 결과, 차세대무선통신미디어 , 
능동형컴퓨팅 (이상 정보통신 테마), 차세대반도체 (전기전자 테마) 및 객체탐지 (센서측정 테마) 기술분야로 
혁신성장 정책금융이 집중되고 있음
[ICT 산업 내 주요 기술분야 혁신성장 정책금융 공급 현황]
                                                            (단위: 억 원, %)
테마
(대분류 )주요 기술분야
(중분류 )정책금융 공급규모 연평균 공급액 
증가율 (%)테마 내 공급 
점유율 (%) 2017 년 말 2021 년 말
정보통신차세대무선통신미디어 7,820 27,865 37.4 45.1
능동형컴퓨팅 352 16,032 159.8 10.1
전기전자 차세대반도체 12,019 53,779 45.4 58.5
센서측정 객체탐지 1,278 6,711 51.4 48.5
▶주요 기술분야별 세부 품목단위로는 5G 이동통신시스템 , 인공지능 (AI), 시스템반도체 및 스마트센서 에 
정책금융 공급량이 높은 것으로 확인됨
￮정부가 미래 먹거리산업으로 선정한 인공지능 (AI)의 미래성장율 (CAGR: 41.0%) 이 가장 높으며 , 시장규모는  
시스템반도체 (3,833.8 억 달러, 2025 년)가 가장 큰 것으로 분석됨
￮4대 품목은 공통적으로 수요기반이 크고, 각국 정부가 중점적으로 육성을 지원하고 있어 시장이 지속 
성장할 것으로 전망되나 , 원천기술 미확보 및 높은 해외 의존도가 약점으로 지적되어 국내 기업의 경쟁력  
강화가 시급한 것으로 평가됨
[혁신성장 ICT 주요 품목 시장전망 ]
                                                            (단위: 억 달러, %)
주요 기술분야
(중분류 )주요 품목
(소분류 )시장규모 전망시장 촉진·저해요인2020 년 2025년(E) CAGR(%)
차세대
무선통신미디어5G이

In [53]:
texts = text_splitter.split_documents(pages)

In [54]:
len(texts)

31

In [57]:
print(texts[1].page_content)

| 2 | CIS이슈리포트 2022-2 호 ▶혁신성장 ICT 산업의 정책금융 공급규모 및 공급속도를 종합적으로 분석한 결과, 차세대무선통신미디어 , 
능동형컴퓨팅 (이상 정보통신 테마), 차세대반도체 (전기전자 테마) 및 객체탐지 (센서측정 테마) 기술분야로 
혁신성장 정책금융이 집중되고 있음
[ICT 산업 내 주요 기술분야 혁신성장 정책금융 공급 현황]
                                                            (단위: 억 원, %)
테마
(대분류 )주요 기술분야
(중분류 )정책금융 공급규모 연평균 공급액 
증가율 (%)테마 내 공급 
점유율 (%) 2017 년 말 2021 년 말
정보통신차세대무선통신미디어 7,820 27,865 37.4 45.1
능동형컴퓨팅 352 16,032 159.8 10.1
전기전자 차세대반도체 12,019 53,779 45.4 58.5
센서측정 객체탐지 1,278 6,711 51.4 48.5
▶주요 기술분야별 세부 품목단위로는 5G 이동통신시스템 , 인공지능 (AI), 시스템반도체 및 스마트센서 에 
정책금융 공급량이 높은 것으로 확인됨
￮정부가 미래 먹거리산업으로 선정한 인공지능 (AI)의 미래성장율 (CAGR: 41.0%) 이 가장 높으며 , 시장규모는  
시스템반도체 (3,833.8 억 달러, 2025 년)가 가장 큰 것으로 분석됨
￮4대 품목은 공통적으로 수요기반이 크고, 각국 정부가 중점적으로 육성을 지원하고 있어 시장이 지속 
성장할 것으로 전망되나 , 원천기술 미확보 및 높은 해외 의존도가 약점으로 지적되어 국내 기업의 경쟁력  
강화가 시급한 것으로 평가됨
[혁신성장 ICT 주요 품목 시장전망 ]
                                                            (단위: 억 달러, %)
주요 기술분야
(중분류 )주요 품목
(소분류 )시장규모 전망시장 촉진·저해요인2020 년 2025년(E) CAGR(%)
차세대


- chunk size 확인

In [58]:
char_list = []
for i in range(len(texts)):
    char_list.append(len(texts[i].page_content))
print(char_list)

[820, 989, 689, 993, 342, 954, 499, 742, 669, 964, 844, 556, 963, 604, 971, 559, 986, 347, 544, 973, 413, 975, 306, 969, 591, 962, 180, 997, 217, 944, 289]


# 기타 Splitter
- 일반적인 글로 된 문서는 모두 textsplitter로 분할 가능
- 그러나, code, latex 등은 처리 불가
- 예를 들어 Python 등은 def, class 등을 하나의 단위로 묶어 분할 필요

In [59]:
from langchain.text_splitter import(
    RecursiveCharacterTextSplitter,
    Language
)

In [60]:
RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON)

['\nclass ', '\ndef ', '\n\tdef ', '\n\n', '\n', ' ', '']

In [61]:
PYTHON_CODE = """
def hello_world():
    print("Hello world")

# Call the function
hello_world()
"""

In [62]:
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size = 50,
    chunk_overlap=0
)

In [63]:
python_docs = python_splitter.create_documents([PYTHON_CODE])

In [64]:
python_docs

[Document(page_content='def hello_world():\n    print("Hello world")'),
 Document(page_content='# Call the function\nhello_world()')]

# 토큰 단위로 텍스트 분할기
- 텍스트 분할의 목적은 LLM이 소화 할 수 있을 정도로 텍스트만 호출 하는 것
- 따라서 LLM이 소화 가능한 크기로 청크 크기를 제한 하는것은 필수적
- LLM은 텍스트를 받아 드릴때 정해진 토큰 이상으로 소화 할 수 없게 설계 되어 있음
- 토큰은 텍스트와 달리 Transformer에서 처리하는 방식에 따라 그 수가 달라질 수 있음
- LLM 앱을 개발한다면 앱에 얹힐 LLM의 토큰 제한을 파악, 해당 LLM이 사용하는 Embedder를 기반으로 토큰 수 계산 해야
- OpenAI의 GTP 모델은 tiktoken 이라는 토크나어저 기반으로 텍스트를 토큰화

In [73]:
import tiktoken

# cl100k_base GPT 모델들에 대해 임베딩시 쓰이는 tokenizer
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [75]:
tiktoken_len(texts[1].page_content), len(texts[1].page_content)

(873, 989)

In [76]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 0,
    length_function = tiktoken_len
)

In [78]:
texts = text_splitter.split_documents(pages)

In [79]:
len(texts[1].page_content), tiktoken_len(texts[1].page_content)

(1068, 960)

- token size 확인

In [82]:
token_list = []
for i in range(len(texts)):
    token_list.append(tiktoken_len(texts[i].page_content))
print(token_list)

[813, 960, 471, 948, 267, 952, 229, 585, 591, 969, 441, 576, 953, 267, 963, 177, 961, 72, 528, 978, 165, 959, 91, 919, 253, 858, 887, 973, 124]
